# Augmented LLM with RAG

In [5]:
from IPython.display import Markdown


In [17]:
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, PromptTemplate

In [18]:
# load llama3.2 model
llm = Ollama(model="llama3.2", request_timeout=120.0)

In [20]:
# load embedding model from huggingface
embed_model = HuggingFaceEmbedding(model_name="nomic-ai/modernbert-embed-base", trust_remote_code=True, cache_folder='./hf_cache')

In [22]:
# setup setting
Settings.llm = llm
Settings.embed_model = embed_model

In [23]:
loader = SimpleDirectoryReader(
    input_dir = "../data",
    required_exts=[".pdf"],
    recursive=True
)
docs = loader.load_data()

In [24]:
# create index from documents
index = VectorStoreIndex.from_documents(docs, show_progress=True)

Generating embeddings: 100%|██████████| 56/56 [00:08<00:00,  6.26it/s]


In [27]:
# create query pattern following
# # https://docs.llamaindex.ai/en/stable/module_guides/models/prompts/usage_pattern/

query_engine = index.as_query_engine(streaming=False)

qa_prompt_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information, "
    "answer the query. In case you don't find information in the context,"
    "answer it anyway with your general knowledge, and don't mention that you"
    "didn't find information in the context\n"
    "Query: {query_str}\n"
    "Answer: "
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

In [31]:
prompt = "what is an agent, how does it differ from agentic workflow?"

response = query_engine.query(prompt)

In [34]:
Markdown(response.response)

According to the text, an "agent" refers to a system where a Large Language Model (LLM) dynamically directs its own processes and tool usage, maintaining control over how it accomplishes tasks. This differs from agentic workflows, which are systems where LLMs and tools are orchestrated through predefined code paths.

In other words, agents have a level of autonomy and self-direction, allowing them to make decisions and adapt to their environment in real-time, whereas agentic workflows follow a more rigid and structured approach.